In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

import pickle
import numpy as np
import pandas as pd

np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_colwidth', None)

In [8]:
with open("./datasets/mnist/train-images-flatten.pkl", "rb") as f:
  train_x = pickle.load(f)

with open("./datasets/mnist/train-labels.pkl", "rb") as f:
  train_t = pickle.load(f)

with open("./datasets/mnist/test-images-flatten.pkl", "rb") as f:
  test_x = pickle.load(f)

with open("./datasets/mnist/test-labels.pkl", "rb") as f:
  test_t = pickle.load(f)

(60000, 784)


In [3]:
class Dataset(data.Dataset):
  def __init__(self, data, label) :
    self.data = data
    self.label = label

  def __len__(self) :
    return len(self.data)

  def __getitem__(self, idx) :
    data = torch.FloatTensor(self.data[idx])
    label = torch.FloatTensor(self.label[idx])

    return data, label

In [4]:
train_dataset = Dataset(train_x, train_t)
test_dataset = Dataset(test_x, test_t)

train_loader = data.DataLoader(train_dataset, batch_size=500, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [5]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(28*28, 100),
      nn.ReLU(),
      nn.Linear(100, 10),
      nn.Softmax(dim=1)
    )

  def forward(self, x):
    return self.layers(x)
  
model = Model()
model

Model(
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
    (3): Softmax(dim=1)
  )
)

In [6]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

for epoch in range(16):
  loss_sm=0
  loss_ct=0

  for x, t in train_loader:
    optimizer.zero_grad()
    y = model(x)
    loss = criterion(y, t)
    loss.backward()
    optimizer.step()

    loss_sm+=loss.item()
    loss_ct+=1

  print('Epoch: %d, Loss: %.3f' % (epoch, loss_sm/loss_ct))

Epoch: 0, Loss: 0.088
Epoch: 1, Loss: 0.076
Epoch: 2, Loss: 0.055
Epoch: 3, Loss: 0.038
Epoch: 4, Loss: 0.029
Epoch: 5, Loss: 0.024
Epoch: 6, Loss: 0.021
Epoch: 7, Loss: 0.019
Epoch: 8, Loss: 0.018
Epoch: 9, Loss: 0.017
Epoch: 10, Loss: 0.016
Epoch: 11, Loss: 0.016
Epoch: 12, Loss: 0.015
Epoch: 13, Loss: 0.015
Epoch: 14, Loss: 0.015
Epoch: 15, Loss: 0.014


In [9]:
model.eval()
total = 0
correct = 0

for x, t in test_loader:
  y = model(x)
  y = torch.argmax(y, dim=1)
  t = torch.argmax(t, dim=1)

  total += t.size(0)
  correct += (y == t).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 91.51%
